In [1]:
!pip install rasterio
!pip install h5py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 13.7 MB/s eta 0:00:00


## 変位用（複数のGeotiffあり）

ファイル読込は２回実施（１回目　抽出したいポイントの座標値（緯度経度）、２回目　グラフに取込みたい変位データのGeotiffファイルを選択（基本は全選択）

In [2]:
# 変位用
import csv
import re
from google.colab import files
import rasterio

# 座標が記載されたテキストファイルのアップロード
uploaded_coord_file = files.upload()
coord_file_name = next(iter(uploaded_coord_file))

# 座標ファイルの読み込み
coordinates = []
with open(coord_file_name, 'r') as coord_file:
    csv_reader = csv.reader(coord_file)
    next(csv_reader)  # 最初の行をスキップする
    for line in csv_reader:
        point, lon, lat = line
        coordinates.append((point.strip(), float(lat.strip()), float(lon.strip())))

print(coordinates)

# GeoTIFFファイルのアップロード
uploaded_tiff_files = files.upload()

# 最初の行のヘッダー名を設定（例: "Point_A_value"）
first_point_header = f'{coordinates[0][0]}_value'

# CSVファイルのヘッダー（2つ目のポイントから始める）
csv_header = ['Date', first_point_header] + [f'{point}_value' for point, _, _ in coordinates[1:]]

# CSVファイルのファイル名
csv_file_name = 'geoTIFF_values.csv'

# 結果をCSVファイルに保存
with open(csv_file_name, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header)  # ヘッダーの書き込み

    for file_name in uploaded_tiff_files.keys():
        row = [re.search(r'\d{8}_\d{8}', file_name).group()]  # 日付の抽出
        for point, latitude, longitude in coordinates:
            with rasterio.open(file_name) as src:
                # 緯度経度をピクセル座標に変換
                r, c = src.index(longitude, latitude)
                # 範囲を超えていないか確認
                r = max(0, min(r, src.height - 1))
                c = max(0, min(c, src.width - 1))
                # 対応するピクセルの値を取得
                value = src.read(1)[r, c]
                row.append(value)
        csv_writer.writerow(row)

# CSVファイルのダウンロード
files.download(csv_file_name)


Saving kobashi_pt.csv to kobashi_pt.csv
[('1', 2902840.0, 375760.0), ('2', 2902800.0, 375760.0), ('3', 2902760.0, 375760.0), ('4', 2902720.0, 375760.0), ('5', 2902680.0, 375760.0), ('6', 2902640.0, 375760.0), ('7', 2902600.0, 375760.0), ('8', 2902560.0, 375760.0), ('9', 2902520.0, 375760.0), ('10', 2902480.0, 375760.0), ('11', 2902440.0, 375760.0), ('12', 2902400.0, 375760.0), ('13', 2902360.0, 375760.0), ('14', 2902320.0, 375760.0), ('15', 2902280.0, 375760.0), ('16', 2902240.0, 375760.0), ('17', 2902200.0, 375760.0), ('18', 2902160.0, 375760.0), ('19', 2902120.0, 375760.0), ('20', 2902080.0, 375760.0), ('21', 2902040.0, 375760.0), ('22', 2902000.0, 375760.0), ('23', 2901960.0, 375760.0), ('24', 2901920.0, 375760.0), ('25', 2901880.0, 375760.0), ('26', 2901840.0, 375760.0), ('27', 2901800.0, 375760.0), ('28', 2901760.0, 375760.0), ('29', 2901720.0, 375760.0), ('30', 2901680.0, 375760.0), ('31', 2901640.0, 375760.0), ('32', 2901600.0, 375760.0), ('33', 2901560.0, 375760.0), ('34', 2901

Saving 20200103_20200103_timeseries_demErr_Meters_unwrapped.tif to 20200103_20200103_timeseries_demErr_Meters_unwrapped.tif
Saving 20200103_20200115_timeseries_demErr_Meters_unwrapped.tif to 20200103_20200115_timeseries_demErr_Meters_unwrapped.tif
Saving 20200103_20200127_timeseries_demErr_Meters_unwrapped.tif to 20200103_20200127_timeseries_demErr_Meters_unwrapped.tif
Saving 20200103_20200208_timeseries_demErr_Meters_unwrapped.tif to 20200103_20200208_timeseries_demErr_Meters_unwrapped.tif
Saving 20200103_20200220_timeseries_demErr_Meters_unwrapped.tif to 20200103_20200220_timeseries_demErr_Meters_unwrapped.tif
Saving 20200103_20200303_timeseries_demErr_Meters_unwrapped.tif to 20200103_20200303_timeseries_demErr_Meters_unwrapped.tif
Saving 20200103_20200315_timeseries_demErr_Meters_unwrapped.tif to 20200103_20200315_timeseries_demErr_Meters_unwrapped.tif
Saving 20200103_20200327_timeseries_demErr_Meters_unwrapped.tif to 20200103_20200327_timeseries_demErr_Meters_unwrapped.tif
Saving 2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 補助的なもの

## ファイルサイズが大きい場合用

In [ ]:
import csv
import re
import os
import rasterio

# 座標ファイルのパス
coord_file_path = '/content/高松グラフ用ポイント.csv'  # ここを座標ファイルのパスに変更

# 座標ファイルの読み込み
coordinates = []
with open(coord_file_path, 'r') as coord_file:
    csv_reader = csv.reader(coord_file)
    next(csv_reader)  # 最初の行をスキップする
    for line in csv_reader:
        point, lon, lat = line
        coordinates.append((point.strip(), float(lat.strip()), float(lon.strip())))

print(coordinates)

# GeoTIFFファイルが保存されているフォルダのパス
tiff_folder_path = '/content/data'  # ここをフォルダのパスに変更

# 最初の行のヘッダー名を設定（例: "Point_A_value"）
first_point_header = f'{coordinates[0][0]}_value'

# CSVファイルのヘッダー（2つ目のポイントから始める）
csv_header = ['Date', first_point_header] + [f'{point}_value' for point, _, _ in coordinates[1:]]

# CSVファイルのファイル名
csv_file_name = 'geoTIFF_values.csv'

# 結果をCSVファイルに保存
with open(csv_file_name, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header)  # ヘッダーの書き込み

    for file_name in os.listdir(tiff_folder_path):
        if file_name.endswith('.tif') or file_name.endswith('.tiff'):
            row = [re.search(r'\d{8}_\d{8}', file_name).group()]  # 日付の抽出
            full_file_path = os.path.join(tiff_folder_path, file_name)
            for point, latitude, longitude in coordinates:
                with rasterio.open(full_file_path) as src:
                    # 緯度経度をピクセル座標に変換
                    r, c = src.index(longitude, latitude)
                    # 範囲を超えていないか確認
                    r = max(0, min(r, src.height - 1))
                    c = max(0, min(c, src.width - 1))
                    # 対応するピクセルの値を取得
                    value = src.read(1)[r, c]
                    row.append(value)
            csv_writer.writerow(row)


[('1', 34.3561287, 134.0548301), ('2', 34.3557087, 134.0548301), ('3', 34.3561287, 134.0552501), ('4', 34.3557087, 134.0552501), ('5', 34.35528871, 134.0552501), ('6', 34.35486871, 134.0552501), ('7', 34.35444871, 134.0552501), ('8', 34.3565487, 134.0556701), ('9', 34.3561287, 134.0556701), ('10', 34.3557087, 134.0556701), ('11', 34.35528871, 134.0556701), ('12', 34.35486871, 134.0556701), ('13', 34.35444871, 134.0556701), ('14', 34.3565487, 134.0560901), ('15', 34.3561287, 134.0560901), ('16', 34.3557087, 134.0560901), ('17', 34.35528871, 134.0560901), ('18', 34.35486871, 134.0560901), ('19', 34.35444871, 134.0560901), ('20', 34.3565487, 134.0565101), ('21', 34.3561287, 134.0565101), ('22', 34.3557087, 134.0565101), ('23', 34.35528871, 134.0565101), ('24', 34.35486871, 134.0565101), ('25', 34.35444871, 134.0565101), ('26', 34.35402872, 134.0565101), ('27', 34.3565487, 134.0569301), ('28', 34.3561287, 134.0569301), ('29', 34.3557087, 134.0569301), ('30', 34.35528871, 134.0569301), ('31

## コヒーレンス用

In [ ]:
# 変位用
import csv
import re
from google.colab import files
import rasterio

# 座標が記載されたテキストファイルのアップロード
uploaded_coord_file = files.upload()
coord_file_name = next(iter(uploaded_coord_file))

# 座標ファイルの読み込み
coordinates = []
with open(coord_file_name, 'r') as coord_file:
    csv_reader = csv.reader(coord_file)
    next(csv_reader)  # 最初の行をスキップする
    for line in csv_reader:
        point, lon, lat = line
        coordinates.append((point.strip(), float(lat.strip()), float(lon.strip())))

print(coordinates)

# GeoTIFFファイルのアップロード
uploaded_tiff_files = files.upload()

# 最初の行のヘッダー名を設定（例: "Point_A_value"）
first_point_header = f'{coordinates[0][0]}_value'

# CSVファイルのヘッダー（2つ目のポイントから始める）
csv_header = ['Date', first_point_header] + [f'{point}_value' for point, _, _ in coordinates[1:]]

# CSVファイルのファイル名
csv_file_name = 'geoTIFF_values.csv'

# 結果をCSVファイルに保存
with open(csv_file_name, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header)  # ヘッダーの書き込み

    for file_name in uploaded_tiff_files.keys():
        # row = [re.search(r'\d{8}_\d{8}', file_name).group()]  # 日付の抽出 使わない！！
        row = [file_name]  # ファイル名を直接使用
        for point, latitude, longitude in coordinates:
            with rasterio.open(file_name) as src:
                # 緯度経度をピクセル座標に変換
                r, c = src.index(longitude, latitude)
                # 範囲を超えていないか確認
                r = max(0, min(r, src.height - 1))
                c = max(0, min(c, src.width - 1))
                # 対応するピクセルの値を取得
                value = src.read(1)[r, c]
                row.append(value)
        csv_writer.writerow(row)

# CSVファイルのダウンロード
files.download(csv_file_name)


## 水面や変位計算不可のデータ削除

In [ ]:
import pandas as pd

# CSVファイルを読み込む（ここではファイルパスを 'your_file.csv' としています）
file_path = '/content/geoTIFF_values.csv'
df = pd.read_csv(file_path)

# 3行目に値がない列を削除
df_filtered_columns = df.loc[:, df.iloc[2] != 0]

# 結果を表示
print(df_filtered_columns.head())

# 結果を新しいCSVファイルに保存する場合
df_filtered_columns.to_csv('filtered_data.csv', index=False)


                Date   4_value   6_value   7_value   8_value   9_value  \
0  20210107_20210107  0.000000  0.000000  0.000000  0.000000  0.000000   
1  20210107_20210119 -0.001157 -0.001650 -0.001103 -0.001359 -0.001612   
2  20210107_20210131 -0.000310 -0.000499 -0.000079 -0.000427 -0.000807   
3  20210107_20210212 -0.004634 -0.004802 -0.004641 -0.004834 -0.004955   
4  20210107_20210224 -0.001773 -0.002029 -0.001622 -0.001935 -0.002002   

   11_value  12_value  13_value  14_value  ...  1464_value  1465_value  \
0  0.000000  0.000000  0.000000  0.000000  ...    0.000000    0.000000   
1 -0.001086 -0.001722 -0.001712 -0.001538  ...   -0.000157   -0.002688   
2 -0.000080 -0.000558 -0.000764 -0.000877  ...    0.003493    0.001112   
3 -0.004491 -0.004726 -0.004873 -0.005010  ...   -0.006867   -0.006414   
4 -0.001639 -0.002017 -0.001968 -0.002045  ...   -0.004474   -0.002660   

   1466_value  1467_value  1468_value  1469_value  1470_value  1472_value  \
0    0.000000    0.000000    0.00

## フィルタで残ったポイントからポイントファイル作成

In [ ]:
import pandas as pd

# CSVファイルを読み込む（ファイルパスは 'your_file.csv' に置き換えてください）
df = pd.read_csv('/content/filtered_data.csv')

# ヘッダー（最初の行）の2列目からの値を取得
header_values = df.columns[1:]  # 1列目を除外

# それぞれのヘッダー値を改行で区切ってCSV形式の文字列として保存
header_csv_string = "\n".join(header_values)

# CSV形式の文字列をファイルに書き出す
with open('header_values.csv', 'w') as file:
    file.write(header_csv_string)
